## Tool Calling

Tool calling is a pattern where an agent uses a tool to perform a task that enables it to take actions in the real world. Examples include:

- An agent using a tool to browse the web to find information.
- An agent using a tool to place an order on a website.
- An agent using a tool to send emails.


###Why Is This Important?

Large Language Models (LLMs) are not able to take actions in the real world. They can only generate text that they have been trained on. Sometimes, this information may be outdated or incorrect or not custom to your application. In such a case, you want to use a tool to retrieve most relevant information and use it to generate a response. Or you want to use a tool to take an action in the real world.

In cases you want to use the LLM to perfect what it can generated, in the last lesson, we saw how to use the Reflection agent to improve the LLM's output. Sometimes, you want the LLM to take an action in the real world. For example, you want to use the LLM to send an email to a client. In such a case, you want to use a tool to send an email.


### How To Use Tool Calling

To achieve tool calling, we simply write a Python function, let's say a function to add two numbers. We then pass this function to the LLM as a tool. The LLM can then use this tool to add two numbers.

Okay, so how do we do this?


#### Step 1: Write a function

Let's start by writing a function to add two numbers.

In [45]:
def add_two_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

NOTE: When writing this functions, make sure to use docstrings to describe the function and type hints to describe the parameters and return type. This will help the LLM understand the function and use it correctly.


### Step 2: Write LLM Prompt Description The Function And How To Use It

In [46]:
function_calling_prompt = """You are an AI assistant with function calling capabilities. Your primary role is to interpret user requests and call appropriate functions when needed.

When presented with function definitions within <tools></tools> XML tags, you should:

1. Analyze the user's request to determine if a function call is necessary
2. Carefully inspect the function signature, paying close attention to parameter types and requirements
3. When calling a function, format your response using the following structure:

<tool_call>
{"name": "function_name", "arguments": {"param1": "value1", "param2": "value2"}}
</tool_call>

Important guidelines:
- Never make assumptions about parameter values - use only information explicitly provided by the user
- Respect parameter types as specified in the function definitions (e.g., string, integer, boolean)
- You may call multiple functions if necessary to fulfill the request
- If missing critical information needed for a function call, ask the user for clarification

Function definitions will be provided in this format:
<tools>
{
    "name": "function_name",
    "description": "Function description",
    "parameters": {
        "properties": {
            "parameter1": {
                "type": "data_type"
            },
            "parameter2": {
                "type": "data_type"
            }
        }
    }
}
</tools>

Ensure your function calls use valid JSON syntax and include all required parameters.
"""

Using this prompt, the LLM will understand and return an output similar to this XML format:

```json
<tool_call>
{"name": "add_two_numbers", "arguments": {"a": 1, "b": 2}}
</tool_call>
```

Now, let's write a prompt to the LLM to use the function we wrote.


In [47]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import re
load_dotenv()

True

In [48]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [49]:
# System prompt for the LLM based on the function_calling_prompt for the add_two_numbers function
system_prompt = """You are an AI assistant with function calling capabilities. Your primary role is to interpret user requests and call appropriate functions when needed.

When presented with function definitions within <tools></tools> XML tags, you should:

1. Analyze the user's request to determine if a function call is necessary
2. Carefully inspect the function signature, paying close attention to parameter types and requirements
3. When calling a function, format your response using the following structure:

<tool_call>
{
    "name": "add_two_numbers",
    "description": "Used to add two numbers together",
    "parameters": {
        "properties": {
            "a": {
                "type": "int"
            },
            "b": {
                "type": "int"
            }
        }
    }
}

Expected output:

<tool_call>
{"name": "add_two_numbers", "arguments": {"a": 1, "b": 2}}
</tool_call>
"""


Now that we have the system prompt, we can use it to call the LLM.

NOTE: Use an LLM that supports function calling.

In [50]:
function_call_prompt = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": "What is the sum of 1 and 2. Use the tool provided to you."
    }
]


response = client.chat.completions.create(
    model="gpt-4o",
    messages=function_call_prompt,
)

In [51]:
response_content = response.choices[0].message.content
print(response_content)


To find the sum of 1 and 2, I'll use a function call.

<tool_call>
{"name": "add_two_numbers", "arguments": {"a": 1, "b": 2}}
</tool_call>


From the query input, I explicitly asked the LLM to use the tool provided to it. This is because the LLM can naturally add the two numbers together without using the tool. I need to explicitly tell it to use the tool.

In [52]:
def transform_response_to_dict(response_content: str) -> dict:
    try:
        match = re.search(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", response_content, re.DOTALL)
        if not match:
            raise ValueError("No <tool_call> JSON found in response.")
        json_str = match.group(1)
        return json.loads(json_str)
    except (json.JSONDecodeError, ValueError) as e:
        return {
            "name": type(e).__name__,
            "message": str(e),
            "stack": None
        }

In [53]:
transformed_response = transform_response_to_dict(response_content)
print(transformed_response)

{'name': 'add_two_numbers', 'arguments': {'a': 1, 'b': 2}}


In [54]:
function_call_response = add_two_numbers(**transformed_response["arguments"])
print(function_call_response)

3


Based on this results from the function call, we can pass the output from here to the LLM in a prompt as an observation and the a more natural language response.



In [56]:
function_call_prompt.append({
    "role": "user",
    "content": f"Observation from tool call: {function_call_response}"
})

response = client.chat.completions.create(
    model="gpt-4o",
    messages=function_call_prompt,
)
response.choices[0].message.content

'It seems like you want to add two numbers using a function tool. I will do that for you.\n\n<tool_call>\n{"name": "add_two_numbers", "arguments": {"a": 1, "b": 2}}\n</tool_call>'

### Building Tool Decorators

In [57]:
from typing import Callable, Dict, Any

def extract_function_metadata(function: Callable) -> Dict[str, Any]:
    """
    Creates a metadata dictionary from a function's signature.
    
    Parameters:
        function: The target function to analyze
        
    Returns:
        A dictionary containing the function's name, documentation, and parameter specifications
    """
    # Initialize the basic structure
    metadata = {
        "name": function.__name__,
        "description": function.__doc__,
        "parameters": {"properties": {}}
    }
    
    # Extract parameter types (excluding return annotation)
    parameter_types = {
        param_name: {"type": param_type.__name__}
        for param_name, param_type in function.__annotations__.items()
        if param_name != "return"
    }
    
    # Add parameters to metadata
    metadata["parameters"]["properties"] = parameter_types
    return metadata

def convert_argument_types(tool_invocation: Dict[str, Any], function_spec: Dict[str, Any]) -> Dict[str, Any]:
    """
    Ensures all arguments match their expected types according to the function specification.
    
    Parameters:
        tool_invocation: Dictionary with the tool name and arguments
        function_spec: Dictionary containing the expected parameter types
        
    Returns:
        Updated tool invocation with correctly typed arguments
    """
    expected_params = function_spec["parameters"]["properties"]
    
    # Type conversion mapping
    type_converters = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float
    }
    
    # Convert each argument to its expected type if needed
    for arg_name, arg_value in tool_invocation["arguments"].items():
        target_type = expected_params[arg_name].get("type")
        if not isinstance(arg_value, type_converters[target_type]):
            tool_invocation["arguments"][arg_name] = type_converters[target_type](arg_value)
            
    return tool_invocation

class Tool:
    """
    Wrapper class that encapsulates a function as a callable tool.
    
    Attributes:
        name: Tool identifier
        function: The underlying function
        specification: JSON-formatted function metadata
    """
    def __init__(self, name: str, function: Callable, specification: str):
        self.name = name
        self.function = function
        self.specification = specification
        
    def __str__(self):
        return self.specification
        
    def execute(self, **kwargs):
        """
        Runs the wrapped function with the provided arguments.
        
        Parameters:
            **kwargs: Arguments to pass to the function
            
        Returns:
            The function's result
        """
        return self.function(**kwargs)

def tool(function: Callable) -> Tool:
    """
    Decorator that transforms a regular function into a Tool instance.
    
    Parameters:
        function: The function to convert into a tool
        
    Returns:
        A fully configured Tool object
    """
    def create_tool():
        metadata = extract_function_metadata(function)
        return Tool(
            name=metadata.get("name"),
            function=function,
            specification=json.dumps(metadata)
        )
    
    return create_tool()

In [60]:
@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """Calculate the area of a rectangle."""
    return length * width

In [90]:
calculate_area_of_rectangle.specification

'{"name": "calculate_area_of_rectangle", "description": "Calculate the area of a rectangle.", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}'

In [61]:
calculate_area_of_rectangle.execute(length=10, width=20)

200

In [64]:
def understanding_tool_decorator():
    # Print tool specifications
    print("Available tools:")
    print(f"1. {calculate_area_of_rectangle}")
    
    # Example tool call (simulating AI model output)
    area_tool_call = {
        "name": "calculate_area",
        "arguments": {
            "length": "10",  # String instead of float to demonstrate type conversion
            "width": 5.5
        }
    }
    
    # Parse the tool specification to get expected types
    area_spec = json.loads(calculate_area_of_rectangle.specification)
    
    # Validate and convert argument types
    validated_call = convert_argument_types(area_tool_call, area_spec)
    
    # Execute the tool
    result = calculate_area_of_rectangle.execute(**validated_call["arguments"])
    print(f"\nCalculated area: {result}")

In [65]:
understanding_tool_decorator()

Available tools:
1. {"name": "calculate_area_of_rectangle", "description": "Calculate the area of a rectangle.", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}

Calculated area: 55.0


Here is a simple explanation of how the tool decorator works:

1. The `@tool` decorator is used to transform a regular function into a Tool instance.
2. The `extract_function_metadata` function is used to extract the function's name, documentation, and parameter specifications.
3. The `convert_argument_types` function is used to ensure all arguments match their expected types according to the function specification.
4. The `Tool` class is used to encapsulate the function as a callable tool.
5. The `execute` method is used to run the wrapped function with the provided arguments.


### Implementing Tool Calling Agent

Now we can move on ahead and implement the tool calling agent. For this, we'll create a Python class that we can pass in tools to.



In [91]:
import json
import re
from typing import List, Dict, Any, Callable

class ToolCallingAgent:
    """
    An agent that integrates language models with function-calling tools.
    
    This class manages the interaction between a language model and a set of tools,
    allowing the model to decide when to call functions and processing the results.
    """
    
    def __init__(self, llm: Callable, system_prompt: str, tools: List[Any]):
        """
        Initialize an AI agent with a language model and tools.
        
        Args:
            llm: Function that takes a model name and messages and returns a response
            system_prompt: Instructions for guiding the language model's behavior
            tools: List of Tool objects created with the @tool decorator
        """
        self.model = llm
        self.system_prompt = system_prompt
        self.tools = {tool.name: tool for tool in tools}
        self.conversation_history = []
        self.max_iterations = 5  # Prevent infinite loops
        
    def format_tools_for_prompt(self) -> str:
        """Format all available tools into a format the language model can understand."""
        tools_json = []
        for tool in self.tools.values():
            try:
                # Use the specification provided by the @tool decorator
                tools_json.append(json.loads(tool.specification))
            except (json.JSONDecodeError, AttributeError):
                # Fallback for tools without proper specification
                tools_json.append({
                    "name": tool.name,
                    "description": getattr(tool, "description", "No description available"),
                    "parameters": {"properties": {}}
                })
                
        return f"<tools>\n{json.dumps(tools_json, indent=2)}\n</tools>"
    
    def extract_tool_calls(self, response: str) -> List[Dict[str, Any]]:
        """
        Parse the language model's response to extract tool call requests.
        
        Args:
            response: The text response from the language model
            
        Returns:
            A list of tool call dictionaries with 'name' and 'arguments' keys
        """
        tool_calls = []
        pattern = r"<tool_call>(.*?)</tool_call>"
        matches = re.findall(pattern, response, re.DOTALL)
        
        for match in matches:
            try:
                tool_call = json.loads(match.strip())
                if "name" in tool_call and "arguments" in tool_call:
                    tool_calls.append(tool_call)
            except json.JSONDecodeError:
                continue
        
        # print(tool_calls)
        
        return tool_calls
    
    def execute_tool(self, tool_call: Dict[str, Any]) -> Any:
        """
        Execute a tool based on the model's request.
        
        Args:
            tool_call: Dictionary with 'name' and 'arguments' for the tool
            
        Returns:
            The result from executing the tool
        """
        tool_name = tool_call.get("name")
        arguments = tool_call.get("arguments", {})
        
        if tool_name not in self.tools:
            return f"Error: Tool '{tool_name}' not found"
            
        tool = self.tools[tool_name]
        
        # Validate and convert argument types using the tool's specification
        try:
            if hasattr(tool, "specification"):
                tool_spec = json.loads(tool.specification)
                validated_args = self.convert_argument_types(
                    {"arguments": arguments}, 
                    tool_spec
                )["arguments"]
                arguments = validated_args
        except (json.JSONDecodeError, AttributeError, KeyError):
            # Continue with original arguments if validation fails
            pass
            
        try:
            # Handle execution based on the tool interface
            # First try the execute method for tools created with the @tool decorator
            if hasattr(tool, "execute"):
                return tool.execute(**arguments)
            # Then try the function attribute which is used by the @tool decorator
            elif hasattr(tool, "function"):
                return tool.function(**arguments)
            # Fall back to run method
            elif hasattr(tool, "run"):
                return tool.run(**arguments)
            # Last resort: call the tool directly if it's callable
            elif callable(tool):
                return tool(**arguments)
            else:
                return f"Error: Tool '{tool_name}' is not callable"
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"
    
    def convert_argument_types(self, tool_call: Dict[str, Any], tool_spec: Dict[str, Any]) -> Dict[str, Any]:
        """
        Convert arguments to their expected types based on tool specification.
        
        Args:
            tool_call: Dictionary containing arguments to convert
            tool_spec: Tool specification with expected types
            
        Returns:
            Updated tool call with properly typed arguments
        """
        if "parameters" not in tool_spec or "properties" not in tool_spec["parameters"]:
            return tool_call
            
        properties = tool_spec["parameters"]["properties"]
        
        # Standard type converters
        type_mapping = {
            "int": int,
            "str": str,
            "bool": bool,
            "float": float,
            "integer": int,
            "string": str,
            "boolean": bool,
            "number": float
        }
        
        for arg_name, arg_value in tool_call["arguments"].items():
            if arg_name in properties and "type" in properties[arg_name]:
                expected_type = properties[arg_name]["type"]
                
                if expected_type in type_mapping:
                    converter = type_mapping[expected_type]
                    try:
                        # Only convert if types don't match
                        if not isinstance(arg_value, converter):
                            tool_call["arguments"][arg_name] = converter(arg_value)
                    except (ValueError, TypeError):
                        # Keep original value if conversion fails
                        pass
                        
        return tool_call
    
    def run(self, user_input: str) -> str:
        # Add user input to conversation history
        self.conversation_history.append({"role": "user", "content": user_input})

        final_response = ""
        iterations = 0
        current_prompt = user_input

        while iterations < self.max_iterations:
            iterations += 1

            # Build the messages list for OpenAI API
            messages = [
                {"role": "system", "content": self.system_prompt + "\n\n" + self.format_tools_for_prompt()}
            ]
            # Add conversation history
            for message in self.conversation_history:
                messages.append({"role": message["role"], "content": message["content"]})

            # If this isn't the first iteration, add the current context
            if iterations > 1:
                messages.append({"role": "assistant", "content": current_prompt})

            # Get response from language model
            response = self.model(model="gpt-4o", messages=messages)
            model_response = response.choices[0].message.content

            # Extract tool calls from response
            tool_calls = self.extract_tool_calls(model_response)
            
            # If no tool calls or reached max iterations, return the response
            if not tool_calls or iterations == self.max_iterations:
                final_response = model_response
                break
                
            # Execute tools and collect results
            results = []
            for tool_call in tool_calls:
                result = self.execute_tool(tool_call)
                results.append({
                    "tool": tool_call.get("name"),
                    "arguments": tool_call.get("arguments"),
                    "result": result
                })
                
            # Format results for next iteration
            current_prompt = "Tool results:\n"
            for res in results:
                result_str = str(res["result"])
                if isinstance(res["result"], (dict, list)):
                    try:
                        result_str = json.dumps(res["result"], indent=2)
                    except:
                        pass
                        
                current_prompt += f"- {res['tool']}{json.dumps(res['arguments'])}: {result_str}\n"
        
        # Add final response to conversation history
        self.conversation_history.append({
            "role": "assistant", 
            "content": final_response
        })
        
        return final_response
        
    def reset_conversation(self):
        """Clear the conversation history."""
        self.conversation_history = []

In [92]:
agent = ToolCallingAgent(llm=client.chat.completions.create, system_prompt=system_prompt, tools=[calculate_area_of_rectangle])

In [93]:
agent.run("What is the area of a rectangle with a length of 10 and a width of 20?")

[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 10.0, 'width': 20.0}}]
[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 10.0, 'width': 20.0}}]
[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 10.0, 'width': 20.0}}]
[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 10.0, 'width': 20.0}}]
[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 10.0, 'width': 20.0}}]


'<tool_call>\n{"name": "calculate_area_of_rectangle", "arguments": {"length": 10.0, "width": 20.0}}\n</tool_call>'

I'll create a diagram that explains how the ToolCallingAgent works and then explain the key components.


## ToolCallingAgent: How It Works

The diagram illustrates how the ToolCallingAgent orchestrates interactions between a language model and tool functions. Here's a breakdown of the key components and process flow:

### Core Components

1. **Language Model (LLM)**
   - The AI system that generates responses and decides when to use tools
   - Receives formatted prompts and returns responses that may include tool calls

2. **Tools Registry**
   - Collection of functions decorated with `@tool`
   - Each tool has a name, specification (JSON schema), and implementation
   - Tools are registered with the agent during initialization

3. **Agent Core Logic**
   - Manages conversation history and context
   - Processes tool calls extracted from LLM responses
   - Validates and converts argument types for tool execution
   - Formats tool results for the LLM

### Execution Flow

1. **User Input Processing**
   - User query is added to conversation history
   - The agent prepares to process the request

2. **LLM Prompt Creation**
   - Tools are formatted into a structured format the LLM can understand
   - System prompt, tools specifications, and conversation history are combined
   - The complete prompt is sent to the language model

3. **Tool Call Extraction & Execution**
   - Agent parses the LLM response for `<tool_call>` tags
   - For each tool call, the agent:
     - Identifies the requested tool
     - Validates and converts argument types
     - Executes the tool and captures results

4. **Result Processing**
   - Tool results are formatted and sent back to the LLM
   - This allows the LLM to incorporate tool outputs into its reasoning

5. **Iterative Reasoning**
   - Steps 2-4 repeat if more tool calls are needed
   - Limited to maximum iterations to prevent infinite loops

6. **Final Response**
   - When no more tool calls are needed, the final response is returned to the user
   - The complete interaction is saved in conversation history

This architecture enables seamless integration between language models and functional tools, allowing the LLM to access external capabilities when needed to fulfill user requests.

![Tool Calling Agent](../../../images/tool_calling_agent.png)

### Custom Class Implementation

Now that we have gone over the basics of how the ToolCallingAgent works, let's implement a custom class that uses the ToolCallingAgent to solve a problem or in other projects.